In [1]:
%%html
<h2>TensorFlow による実装 ：手書きの数字の画像データ判定<br>
活性化関数：Parametric ReLU<br>
隠れ層：3層<br>
</h2>

In [2]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(123)

'''
データの生成
'''
mnist = datasets.fetch_mldata('MNIST original', data_home='.')

n = len(mnist.data)
N = 10000  # MNISTの一部を使う
train_size = 0.8
indices = np.random.permutation(range(n))[:N]  # ランダムにN枚を選択

X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)]  # 1-of-K 表現に変換

X_train, X_test, Y_train, Y_test =\
    train_test_split(X, Y, train_size=train_size)

'''
モデル設定
'''
n_in = len(X[0])  # 784
n_hidden = 200
n_out = len(Y[0])  # 10


def prelu(x, alpha):
    return tf.maximum(tf.zeros(tf.shape(x)), x) \
        + alpha * tf.minimum(tf.zeros(tf.shape(x)), x)


x = tf.placeholder(tf.float32, shape=[None, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])

# 入力層 - 隠れ層
W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
alpha0 = tf.Variable(tf.zeros([n_hidden]))
h0 = prelu(tf.matmul(x, W0) + b0, alpha0)

# 隠れ層 - 隠れ層
W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
alpha1 = tf.Variable(tf.zeros([n_hidden]))
h1 = prelu(tf.matmul(h0, W1) + b1, alpha1)

W2 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_hidden]))
alpha2 = tf.Variable(tf.zeros([n_hidden]))
h2 = prelu(tf.matmul(h1, W2) + b2, alpha2)

W3 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b3 = tf.Variable(tf.zeros([n_hidden]))
alpha3 = tf.Variable(tf.zeros([n_hidden]))
h3 = prelu(tf.matmul(h2, W3) + b3, alpha3)

# 隠れ層 - 出力層
W4 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b4 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h3, W4) + b4)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y),
                               reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

'''
モデル学習
'''
epochs = 50
batch_size = 200

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

n_batches = (int)(N * train_size) // batch_size

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end]
        })

    # 訓練データに対する学習の進み具合を出力
    loss = cross_entropy.eval(session=sess, feed_dict={
        x: X_,
        t: Y_
    })
    acc = accuracy.eval(session=sess, feed_dict={
        x: X_,
        t: Y_
    })
    print('epoch:', epoch, ' loss:', loss, ' accuracy:', acc)

'''
予測精度の評価
'''
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test
})
print('accuracy: ', accuracy_rate)

# 学習後の alpha の値を確認
print(sess.run(alpha0))
print(sess.run(alpha1))
print(sess.run(alpha2))
print(sess.run(alpha3))

epoch: 0  loss: 2.30228  accuracy: 0.117625
epoch: 1  loss: 2.30184  accuracy: 0.11525
epoch: 2  loss: 2.30136  accuracy: 0.11525
epoch: 3  loss: 2.30083  accuracy: 0.11525
epoch: 4  loss: 2.30017  accuracy: 0.11525
epoch: 5  loss: 2.29934  accuracy: 0.11525
epoch: 6  loss: 2.29821  accuracy: 0.11675
epoch: 7  loss: 2.2965  accuracy: 0.130875
epoch: 8  loss: 2.29363  accuracy: 0.193125
epoch: 9  loss: 2.28792  accuracy: 0.2945
epoch: 10  loss: 2.27343  accuracy: 0.23625
epoch: 11  loss: 2.21921  accuracy: 0.11975
epoch: 12  loss: 2.03735  accuracy: 0.2555
epoch: 13  loss: 1.53864  accuracy: 0.384875
epoch: 14  loss: 1.18119  accuracy: 0.55975
epoch: 15  loss: 0.958215  accuracy: 0.65125
epoch: 16  loss: 0.737435  accuracy: 0.77025
epoch: 17  loss: 0.634292  accuracy: 0.801
epoch: 18  loss: 0.561059  accuracy: 0.832375
epoch: 19  loss: 0.574404  accuracy: 0.816375
epoch: 20  loss: 0.476518  accuracy: 0.853
epoch: 21  loss: 0.42995  accuracy: 0.874125
epoch: 22  loss: 0.405787  accuracy: